## 6.6 Control Flow

### 6.6.1

> Add rules to the syntax-directed definition of Fig. 6.36 for the following control-flow constructs:

> a) A repeat-statement $\mathbf{repeat}~S~\mathbf{while}~B$.

$S~\rightarrow~\mathbf{repeat}~S_1~\mathbf{while}~B$

$$\begin{array}{lll}
begin &=& newlabel() \\
B.true &=& begin \\
B.false &=& S.next \\
S_1.next &=& newlabel() \\
S.code &=& label(begin)~||~S_1.code~||~\\
& & label(S_1.next)~||~B.code
\end{array}$$

> b) A for-loop $\mathbf{for}~(S_1;~B;~S_2)~S_3$.

$S~\rightarrow~\mathbf{for}~(S_1;~B;~S_2)~S_3$

$$\begin{array}{lll}
S_1.next &=& newlabel() \\
B.true &=& newlabel() \\
B.false &=& S.next \\
S_2.next &=& S_1.next \\
S_3.next &=& newlabel() \\
S.code &=& S_1.code~||~ \\
& & label(S_1.next)~||~B.code~||~\\
& & label(B.true)~||~S_3.code~||~\\
& & label(S_3.next)~||~S_2.code~||~\\
& & gen(\text{'goto'}~S_1.next)
\end{array}$$

### 6.6.2

> Modern machines try to execute many instructions at the same time, including branching instructions. Thus, there is a severe cost if the machine speculatively follows one branch, when control actually goes another way. It is therefore desirable to minimize the number of branches. Notice that the implementation of a while-loop in Fig. 6.35(c) has two branches per iteration: one to enter the body from the condition $B$ and the other to jump back to the code for $B$. As a result, it is usually preferable to implement $\mathbf{while}~(B)~S$ as if it were $\mathbf{if}~B~\{~\mathbf{repeat}~S~\mathbf{until}~!(B)~\}$. Show what the code layout looks like for this translation, and revise the rule for while-loops in Fig. 6.36.

Layout:

$$\begin{array}{c}
B.code \\
B.true \\
S_1.code \\
S_1.next \\
B.code \\
S.next \\
\end{array}$$

Rule:

$S~\rightarrow~\mathbf{if}~B~\{~\mathbf{repeat}~S~\mathbf{until}~!(B)~\}$

$$\begin{array}{lll}
B.true &=& newlabel() \\
B.false &=& S.next \\
S_1.next &=& newlabel() \\
S.code &=& B.code~||~\\
& & label(B.true)~||~S_1.code~||~\\
& & label(S_1.next)~||~B.code
\end{array}$$

### 6.6.3

> Suppose that there were an "exclusive-or" in C. Write the rule for this operator in the style of Fig. 6.37.

$B~\rightarrow~\mathbf{if}~(B_1)~!B_{2t}~\mathbf{else}~B_{2f}$

$$\begin{array}{lll}
B_1.true &=& newlabel() \\
B_1.false &=& newlabel() \\
B_{2t}.true &=& B.false \\
B_{2t}.false &=& B.true \\
B_{2f}.true &=& B.true \\
B_{2f}.false &=& B.false \\
B.code &=& B_1.code~||~\\
& & label(B_1.true)~||~B_{2t}.code~||~\\
& & label(B_1.false)~||~B_{2f}.code \\
\end{array}$$

### 6.6.4

> Translate the following expressions using the goto-avoiding translation scheme of Section 6.6.5:

> a) `if (a==b && c==d || e==f) x == 1;`;

$$\begin{array}{rl}
& \text{ifFalse a==b goto L3} \\
& \text{if c==d goto L2} \\
L3: & \text{ifFalse e==f goto L1} \\
L2: & x == 1 \\
L1: & \\
\end{array}$$

> b) `if (a==b || c==d || e==f) x == 1;`;

$$\begin{array}{rl}
& \text{if a==b goto L2} \\
& \text{if c==d goto L2} \\
& \text{ifFalse e==f goto L1} \\
L2: & x == 1\\
L1: & \\
\end{array}$$

> c) `if (a==b && c==d && e==f) x == 1;`;

$$\begin{array}{rl}
& \text{ifFalse a==b goto L1} \\
& \text{ifFalse c==d goto L1} \\
& \text{ifFalse e==f goto L1} \\
L2: & x == 1\\
L1: & \\
\end{array}$$

### 6.6.5

> Give a translation scheme based on the syntax-directed definition in Figs. 6.36 and 6.37.

### 6.6.6

> Adapt the semantic rules in Figs. 6.36 and 6.37 to allow control to fall through, using rules like the ones in Figs. 6.39 and 6.40.

| PRODUCTION | SEMANTIC RULES |
|:-----------|:---------------|
| $P \rightarrow S$ | $\begin{array}{lll} S.next &=& fall \\ P.code &=& S.code \end{array}$ |
| $S \rightarrow \mathbf{assign}$ | $\begin{array}{lll} S.code &=& \mathbf{assign}.code \end{array}$ |
| $S \rightarrow \mathbf{if}~(~B~)~S_1$ | $\begin{array}{lll} B.true &=& fall \\ B.false &=& S.next \\ S_1.next &=& S.next \\ S.code &=& B.code~||~S_1.code \end{array}$ |
| $S \rightarrow \mathbf{if}~(~B~)~S_1~\mathbf{else}~S_2$ | $\begin{array}{lll} B.true &=& fall \\ B.false &=& newlabel() \\ S_1.next &=& S.next \\ S_2.next &=& S.next \\ S.code &=& B.code~||~S_1.code~||~\\ & & gen(\text{'goto'}~S_1.next)~||~\\& &S_2.code \end{array}$ |
| $S \rightarrow \mathbf{while}~(~B~)~S_1$ | $\begin{array}{lll} begin &=& newlabel() \\ B.true &=& fall \\ B.false &=& S.next \\ S_1.next &=& begin \\ S.code &=& label(begin)~||~B.code~||~ \\& & S_1.code~||~gen(\text{'goto'}~begin) \end{array}$ |
| $S \rightarrow S_1~S_2$ | $\begin{array}{lll} S_1.next &=& fall \\ S_2.next &=& S.next \\ S.code &=& S_1.code~||~S_2.code \end{array}$ |
| | |
| $B \rightarrow B_1~||~B_2$ | $\begin{array}{lll} B_1.true &=& \mathbf{if}~ B.true \ne fall ~\mathbf{then}~ B.true ~\mathbf{else}~ newlabel() \\ B_1.false &=& fall \\ B_2.true &=& B.true \\ B_2.false &=& B.false \\ B.code &=& \mathbf{if}~ B.true \ne fall ~\mathbf{then}~ B_1.code ~||~ B_2.code \\ & & \mathbf{else}~ B_1.code~||~B_2.code~||~label(B_1.true) \end{array}$ |
| $B \rightarrow B_1~\&\&~B_2$ | $\begin{array}{lll} B_1.true &=& fall \\ B_1.false &=& \mathbf{if}~ B.false \ne fall ~\mathbf{then}~ B.true ~\mathbf{else}~ newlabel() \\ B_2.true &=& B.true \\ B_2.false &=& B.false \\ B.code &=& \mathbf{if}~ B.false \ne fall ~\mathbf{then}~ B_1.code ~||~ B_2.code \\ & & \mathbf{else}~ B_1.code~||~B_2.code~||~label(B_1.true) \end{array}$ |
| $B \rightarrow !~B_1$ | $\begin{array}{lll} B_1.true &=& B.false \\ B_1.false &=& B.true \\ B.code &=& B_1.code \end{array}$ |
| $B \rightarrow E_1~\mathbf{rel}~E_2$ | $\begin{array}{lll} test &=& E_1.addr ~\mathbf{rel}~.op E_2.addr \\ s &=& \mathbf{if}~B.true \ne fall ~\mathbf{and}~ B.false \ne fall ~\mathbf{then} \\& & ~~~~gen(\text{'if'}~test~\text{'goto'}~B.true)~||~gen(\text{'goto'}~B.false) \\& & \mathbf{else}~\mathbf{if}~B.true \ne fall ~\mathbf{then}~ gen(\text{'if'}~test~\text{'goto'}~B.true) \\& & \mathbf{else}~\mathbf{if}~B.false \ne fall ~\mathbf{then}~ gen(\text{'ifFalse'}~test~\text{'goto'}~B.false) \\& &\mathbf{else}~\text{''} \\B.code &=& E_1.code~||~E_2.code~||~s \end{array}$ |
| $B \rightarrow \mathbf{true}$ | $\begin{array}{lll} B.code &=& gen(\text{'goto'}~B.true) \end{array}$ |
| $B \rightarrow \mathbf{false}$ | $\begin{array}{lll} B.code &=& gen(\text{'goto'}~B.false) \end{array}$ |

### 6.6.7

> The semantic rules for statements in Exercise 6.6.6 generate unnecessary labels. Modify the rules for statements in Fig.  6.36 to create labels as needed, using a special label _deferred_ to mean that a label has not yet been created. Your rules must generate code similar to that in Example 6.21.

### 6.6.8

> Section 6.6.5 talks about using fall-through code to minimize the number of jumps in the generated intermediate code. However, it does not take advantage of the option to replace a condition by its complement, e.g., replace `if a<b goto L1; goto L2` by `if a>=b goto L2; goto L1`. Develop a SDD that does take advantage of this option when needed.